In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')

In [2]:
pop_movies = pd.read_csv('/home/saddie/Documents/CIS5570/movie_dataset/movies_metadata_pop_movies.csv')
pop_movies = pop_movies.set_index('movie_id')
pop_movies.shape

(2129, 23)

In [3]:
pop_movies['year'] = pd.to_datetime(pop_movies['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
pop_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
movie_id,,,,,,,,,,,,,,,,,,,,,
5,False,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,...,4300000.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6,539,1995
11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,...,775398007.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8,6778,1977
12,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://movies.disney.com/finding-nemo,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",25.497794,...,940335536.0,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There are 3.7 trillion fish in the ocean. They...,Finding Nemo,False,7,6292,2003
13,False,NaN,55000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,48.307194,...,677945399.0,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The world will never be the same, once you've ...",Forrest Gump,False,8,8147,1994
14,False,NaN,15000000,"[{'id': 18, 'name': 'Drama'}]",http://www.dreamworks.com/ab/,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",20.726578,...,356296601.0,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Look closer.,American Beauty,False,7,3438,1999


In [4]:
pop_movies = pop_movies.drop(columns=['release_date'])
pop_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
movie_id,,,,,,,,,,,,,,,,,,,,,
5,False,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,...,4300000.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6,539,1995
11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,...,775398007.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8,6778,1977
12,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://movies.disney.com/finding-nemo,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",25.497794,...,940335536.0,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There are 3.7 trillion fish in the ocean. They...,Finding Nemo,False,7,6292,2003
13,False,NaN,55000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,48.307194,...,677945399.0,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The world will never be the same, once you've ...",Forrest Gump,False,8,8147,1994
14,False,NaN,15000000,"[{'id': 18, 'name': 'Drama'}]",http://www.dreamworks.com/ab/,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",20.726578,...,356296601.0,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Look closer.,American Beauty,False,7,3438,1999


In [5]:
pop_movies['vote_count'] = pop_movies['vote_count'].astype('int')
pop_movies['vote_average'] = pop_movies['vote_average'].astype('int')

In [6]:
pop_movies['genres'] = pop_movies['genres'].fillna('[]').apply(literal_eval)
pop_movies['genres'] = pop_movies['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [7]:
#pop_movies['genres'] = pop_movies['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [8]:
pop_movies

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
movie_id,,,,,,,,,,,,,,,,,,,,,
5,False,NaN,4000000,"[Crime, Comedy]",NaN,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,...,4300000.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6,539,1995
11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[Adventure, Action, Science Fiction]",http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,...,775398007.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8,6778,1977
12,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000,"[Animation, Family]",http://movies.disney.com/finding-nemo,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",25.497794,...,940335536.0,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There are 3.7 trillion fish in the ocean. They...,Finding Nemo,False,7,6292,2003
13,False,NaN,55000000,"[Comedy, Drama, Romance]",NaN,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,48.307194,...,677945399.0,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The world will never be the same, once you've ...",Forrest Gump,False,8,8147,1994
14,False,NaN,15000000,[Drama],http://www.dreamworks.com/ab/,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",20.726578,...,356296601.0,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Look closer.,American Beauty,False,7,3438,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401104,False,NaN,0,[Drama],https://www.netflix.com/title/80171659,tt5541240,en,To the Bone,A young woman dealing with anorexia meets an u...,9.128637,...,0.0,107.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,To the Bone,False,7,714,2017
403119,False,NaN,5500000,"[Drama, Thriller, Horror, Adventure]",https://47metersdownmovie.com/,tt2932536,en,47 Meters Down,Two sisters on Mexican vacation are trapped in...,52.854103,...,44235023.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No way out. No way up. No chance in hell.,47 Meters Down,False,5,548,2017
406990,False,NaN,0,"[Science Fiction, Thriller]",NaN,tt1536537,en,What Happened to Monday,In a world where families are limited to one c...,60.581223,...,0.0,123.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Seven sisters. One identity.,What Happened to Monday,False,7,598,2017


In [9]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+474.) * R) + (474./(474.+v) * 6.0008)

In [10]:
pop_movies['weighted_ratings'] = pop_movies.apply(weighted_rating, axis=1)

In [11]:
pop_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,weighted_ratings
movie_id,,,,,,,,,,,,,,,,,,,,,
5,False,NaN,4000000,"[Crime, Comedy]",NaN,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,...,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6,539,1995,6.000374
11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[Adventure, Action, Science Fiction]",http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,...,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8,6778,1977,7.869330
12,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000,"[Animation, Family]",http://movies.disney.com/finding-nemo,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",25.497794,...,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There are 3.7 trillion fish in the ocean. They...,Finding Nemo,False,7,6292,2003,6.930000
13,False,NaN,55000000,"[Comedy, Drama, Romance]",NaN,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,48.307194,...,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The world will never be the same, once you've ...",Forrest Gump,False,8,8147,1994,7.890080
14,False,NaN,15000000,[Drama],http://www.dreamworks.com/ab/,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",20.726578,...,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Look closer.,American Beauty,False,7,3438,1999,6.878931


In [12]:
pop_movie_genre= pop_movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
pop_movie_genre.name = 'genre'
pop_movie_genre_sep = pop_movies.drop('genres', axis=1).join(pop_movie_genre)
pop_movie_genre_sep

,adult,belongs_to_collection,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,weighted_ratings,genre
movie_id,,,,,,,,,,,,,,,,,,,,,
5,False,NaN,4000000,NaN,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6,539,1995,6.000374,Crime
5,False,NaN,4000000,NaN,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,/eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6,539,1995,6.000374,Comedy
11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,/btTdmkgIvOi0FFip1sPuZI2oQG6.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8,6778,1977,7.869330,Adventure
11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,/btTdmkgIvOi0FFip1sPuZI2oQG6.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8,6778,1977,7.869330,Action
11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,/btTdmkgIvOi0FFip1sPuZI2oQG6.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8,6778,1977,7.869330,Science Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417678,False,NaN,10000000,http://everythingeverythingmovie.com,tt5001718,en,"Everything, Everything",A teenager who's lived a sheltered life becaus...,12.108048,/c8W3Go48Mw0GoNXsGK4W9hNO4Vf.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Risk everything... for love.,"Everything, Everything",False,7,670,2017,6.585996,Drama
417678,False,NaN,10000000,http://everythingeverythingmovie.com,tt5001718,en,"Everything, Everything",A teenager who's lived a sheltered life becaus...,12.108048,/c8W3Go48Mw0GoNXsGK4W9hNO4Vf.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Risk everything... for love.,"Everything, Everything",False,7,670,2017,6.585996,Romance
419430,False,NaN,5000000,http://www.getoutfilm.com/,tt5052448,en,Get Out,Chris and his girlfriend Rose go upstate to vi...,36.894806,/1SwAVYpuLj8KsHxllTF8Dt9dSSX.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Just because you're invited, doesn't mean you'...",Get Out,False,7,2978,2017,6.862798,Mystery


In [13]:
links_small = pd.read_csv('./links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
links_small.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [14]:
smd = pop_movies[pop_movies.index.isin(links_small)]
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,weighted_ratings
movie_id,,,,,,,,,,,,,,,,,,,,,
5,False,NaN,4000000,"[Crime, Comedy]",NaN,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,9.026586,...,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6,539,1995,6.000374
11,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[Adventure, Action, Science Fiction]",http://www.starwars.com/films/star-wars-episod...,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,42.149697,...,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,False,8,6778,1977,7.869330


In [15]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [16]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [17]:
tfidf_matrix.shape

(1887, 65356)

In [19]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.       , 0.       , 0.       , ..., 0.0027905, 0.       ,
       0.       ])

In [20]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [21]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [22]:
get_recommendations('The Godfather').head(10)

104            The Godfather: Part II
1608                       The Family
1224                          Machete
106           The Godfather: Part III
1765                    Run All Night
613                 Road to Perdition
1360    Elite Squad: The Enemy Within
1530                         Sinister
585                 The Addams Family
542                Death at a Funeral
Name: title, dtype: object